# Decorator Utilites

In [1]:
# | default_exp decorators

In [2]:
# | export
from functools import wraps
from torch_snippets.inspector import inspect
from torch_snippets.logger import Info
import time

In [32]:
# | export


def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        output = func(*args, **kwargs)
        end = time.time()
        Info(f"{func.__name__} took {end-start:.2f} seconds to execute")
        return output

    return wrapper


def io(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        if len(args) != 0:
            inspect(args, names=["inputs:args"])
        if kwargs != {}:
            inspect(kwargs, names=["inputs:kwargs"])
        output = func(*args, **kwargs)
        inspect(output, names=["outputs"])
        return output

    return wrapper


def check_kwargs_not_none(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        for key, value in kwargs.items():
            if value is None:
                raise ValueError(f"Input argument '{key}' cannot be None")
        return func(*args, **kwargs)

    return wrapper

In [33]:
@timeit
@io
def foo(a, b):
    import time

    time.sleep(1)
    return a + b


foo(10, 11)

══════════════════════════════════════════════════════════════════

INPUTS:ARGS:
tuple of 2 items

int: 10

int: 11

══════════════════════════════════════════════════════════════════

══════════════════════════════════════════════════════════════════

OUTPUTS:
int: 21

══════════════════════════════════════════════════════════════════

[04/25/23 13:33:51] INFO     foo took 1.02 seconds to execute                               ]8;id=349502;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_13405/2022534928.py\2022534928.py]8;;\:]8;id=754939;file:///var/folders/cp/1fbgq2n922j8ztdsq6551vldkr5sdy/T/ipykernel_13405/2022534928.py#wrapper:9\wrapper:9]8;;\

21

In [34]:
@check_kwargs_not_none
@io
def foo(*, a=None, b=None):
    return a + b


foo(a=None, b=10)

ValueError: Input argument 'a' cannot be None

In [35]:
import nbdev

nbdev.nbdev_export()